In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import anndata as ad
from adjustText import adjust_text

from cellassign import assign_cats
from cellbender.remove_background.downstream import load_anndata_from_input_and_output as load_anndata_cellbender

import gc

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import font_manager, rcParams

import numpy as np

import pandas as pd

import scanpy as sc
import scanpy.external as sce

import scipy.sparse as sparse
from scipy.sparse import csr_matrix

import seaborn as sns

import triku as tk

In [ ]:
# Palettes for UMAP gene expression

magma = [plt.get_cmap('magma')(i) for i in np.linspace(0,1, 80)]
magma[0] = (0.88, 0.88, 0.88, 1)
magma = mpl.colors.LinearSegmentedColormap.from_list("", magma[:65])

In [ ]:
sc.settings.set_figure_params(dpi=100) 
seed = 0

In [ ]:
# Alevin outputs Ensembl IDs, and we will transform those to Gene symbols

from pybiomart import Server

server = Server(host='http://www.ensembl.org')

df = server.marts['ENSEMBL_MART_ENSEMBL'].datasets['mmusculus_gene_ensembl'].query(attributes=['ensembl_gene_id', 'external_gene_name'])
dict_ensemble_gene = dict(zip(df['Gene stable ID'], df['Gene name']))

In [ ]:
def set_plotting_style():
    # Set font and plot properties
    sns.set_style("white")

    font_path = "src/fonts/texgyreheros-regular.otf" # Alternative
    font_path = "/usr/share/texmf/fonts/opentype/public/tex-gyre/texgyreheros-regular.otf"

    mpl.rcParams.update({'font.size': 22})


    # Path to the Helvetica font file
    custom_font = font_manager.FontProperties(fname=font_path)

    # Add the font to Matplotlib's font manager
    font_manager.fontManager.addfont(font_path)

    # Set the font globally
    rcParams['font.family'] = custom_font.get_name()
    rcParams['font.size'] = 16
    rcParams['figure.dpi'] = 200

set_plotting_style()

In [ ]:
def plot_volcano(adata, cluster, pval_threshold=0.0001, lfc_threshold=2, topn=10, bottomn=8, zero_pval='auto', return_df=False, plot_positive_only=True):
    df_pvals = pd.DataFrame({'gene': adata.uns['rank_genes_groups']['names'][cluster], 
                         'adj_pval': adata.uns['rank_genes_groups']['pvals_adj'][cluster], 
                         'logfoldchanges': adata.uns['rank_genes_groups']['logfoldchanges'][cluster],})

    # adjust p values that are zero to a small number
    if zero_pval == 'auto':
        min_nonzero = df_pvals.loc[df_pvals['adj_pval'] > 0, 'adj_pval'].min()
        zero_pval = min_nonzero * 0.1

    df_pvals.loc[df_pvals['adj_pval'] == 0, 'adj_pval'] = zero_pval

    df_pvals['neg_log_pval'] = -np.log10(df_pvals['adj_pval'])

    if plot_positive_only:
        df_pvals = df_pvals[df_pvals['logfoldchanges'] > 0]

    plt.figure(figsize=(5, 4))
    plt.scatter(df_pvals['logfoldchanges'], df_pvals['neg_log_pval'], color='gray', alpha=0.5, s=3)


    significant = (df_pvals['adj_pval'] < pval_threshold) & (abs(df_pvals['logfoldchanges']) > lfc_threshold)
    plt.scatter(df_pvals['logfoldchanges'][significant], df_pvals['neg_log_pval'][significant], color='red', alpha=0.7, s=3)


    df_pvals['pvalxlfc'] = df_pvals['neg_log_pval'] * df_pvals['logfoldchanges']
    top_genes = df_pvals.nlargest(topn, 'pvalxlfc')
    bottom_genes = df_pvals.nsmallest(bottomn, 'pvalxlfc')

    texts = []
    for _, row in pd.concat([top_genes, bottom_genes]).iterrows():
        texts.append(plt.text(row['logfoldchanges'], row['neg_log_pval'], row['gene'], 
                            fontsize=7, ha='center', color='black', 
                            )) 


    adjust_text(texts, 
                arrowprops=dict(arrowstyle='-', color='gray', lw=0.8)) 

    plt.axhline(y=-np.log10(pval_threshold), color='#232323', linestyle='--', linewidth=0.8)
    plt.axvline(x=lfc_threshold, color='#232323', linestyle='--', linewidth=0.8)

    if not plot_positive_only:
        plt.axvline(x=-lfc_threshold, color='#232323', linestyle='--', linewidth=0.8)

    plt.xlabel('LFC')
    plt.ylabel('-log$_{10}$(Adjusted p-value)')
    plt.title('')

    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)


    # plt.savefig('../../../figures/4E_volcano_glia.png', dpi=300, bbox_inches='tight')

    plt.show()

    if return_df:
        return df_pvals

In [ ]:
def preprocessing_adata_sub(adata_sub, integrate = True, k=None):
    if k is None:
        k = int(0.5 * len(adata_sub) ** 0.5)

    if integrate:
        use_rep = 'X_harmony'
    else:
        use_rep = 'X_pca'

    sc.pp.filter_cells(adata_sub, min_counts=10)
    sc.pp.filter_genes(adata_sub, min_counts=20)
    sc.pp.pca(adata_sub, n_comps=15, random_state=seed, use_highly_variable=False)
    if integrate:
        sce.pp.harmony_integrate(adata_sub, 'batch', random_state=seed,
                                    basis='X_pca', adjusted_basis='X_harmony', max_iter_harmony=30, verbose=False)
        
    sc.pp.neighbors(adata_sub, n_neighbors=k, random_state=seed, metric='correlation', 
                    use_rep=use_rep)
    tk.tl.triku(adata_sub, use_raw=False)

    sc.pp.pca(adata_sub, n_comps=15, random_state=seed, use_highly_variable=True)
    if integrate:
        sce.pp.harmony_integrate(adata_sub, 'batch', random_state=seed,
                                basis='X_pca', adjusted_basis='X_harmony', max_iter_harmony=30, verbose=False)
    sc.pp.neighbors(adata_sub, n_neighbors=k, random_state=seed, metric='correlation', 
                    use_rep=use_rep)


# Data loading and QC

In [ ]:
def load_mtx(mtx_path, barcodes_path, varnames_path):
    """
    Load a matrix from a .mtx file and its associated barcodes and variable names.
    """
    adata = sc.read_mtx(mtx_path).T
    adata_barcodes = pd.read_csv(barcodes_path, header=None, sep='\t')
    adata_varnames = pd.read_csv(varnames_path, header=None, sep='\t')
    
    adata.var_names = adata_varnames[0].values
    adata.obs_names = adata_barcodes[0].values
    
    return adata

In [ ]:
def load_full_adata(STARsolo_path, raw_counts=True, include_cellbender=True, load_velocyto=True):

    if raw_counts:
        dirsub = 'raw'
    else:
        dirsub = 'filtered'

    if include_cellbender: # This should be run on a second pass, the first pass should be false
        adata = load_anndata_cellbender(
                    input_file=f'{STARsolo_path}/adata_raw.h5ad',
                    output_file=f'{STARsolo_path}/CellBender/adata_raw_cellbender.h5',
                    input_layer_key='raw_counts',  # this will be the raw data layer
                )
        
        adata.raw = adata
    else:
        adata = load_mtx(f'{STARsolo_path}/GeneFull/{dirsub}/UniqueAndMult-EM.mtx', f'{STARsolo_path}/GeneFull/{dirsub}/barcodes.tsv', f'{STARsolo_path}/GeneFull/{dirsub}/features.tsv')
        adata.layers['raw_counts'] = adata.X.copy()

    if load_velocyto:
        adata_velocyto_spliced = load_mtx(f'{STARsolo_path}/Velocyto/{dirsub}/spliced.mtx', f'{STARsolo_path}/Velocyto/{dirsub}/barcodes.tsv', f'{STARsolo_path}/Velocyto/{dirsub}/features.tsv')
        adata_velocyto_unspliced = load_mtx(f'{STARsolo_path}/Velocyto/{dirsub}/unspliced.mtx', f'{STARsolo_path}/Velocyto/{dirsub}/barcodes.tsv', f'{STARsolo_path}/Velocyto/{dirsub}/features.tsv')
        adata_velocyto_ambiguous = load_mtx(f'{STARsolo_path}/Velocyto/{dirsub}/ambiguous.mtx', f'{STARsolo_path}/Velocyto/{dirsub}/barcodes.tsv', f'{STARsolo_path}/Velocyto/{dirsub}/features.tsv')

        adata.layers['spliced'] = adata_velocyto_spliced[adata.obs_names, adata.var_names].X.copy()
        adata.layers['unspliced'] = adata_velocyto_unspliced[adata.obs_names, adata.var_names].X.copy()
        adata.layers['ambiguous'] = adata_velocyto_ambiguous[adata.obs_names, adata.var_names].X.copy()

        del adata_velocyto_spliced, adata_velocyto_unspliced, adata_velocyto_ambiguous
    
    adata.var_names_make_unique()

    return adata

In [ ]:
adata_AZ7845 = load_full_adata('data/ARAUZO_03/results_STAR/AZ7845/Solo.out', include_cellbender=False, load_velocyto=False)

In [ ]:
adata_AZ7846 = load_full_adata('data/ARAUZO_03/results_STAR/AZ7846/Solo.out', include_cellbender=False, load_velocyto=False)

## CellBender (remove ambient RNA)

In [ ]:
adata_AZ7845.write_h5ad('data/ARAUZO_03/results_STAR/AZ7845/Solo.out/adata_raw.h5ad')

In [ ]:
!docker run --gpus all --rm -v /mnt/data:/mnt/data \
  us.gcr.io/broad-dsde-methods/cellbender:0.3.2 \
  cellbender remove-background \
    --input /mnt/data/Proyectos/kranocito/data/ARAUZO_03/results_STAR/AZ7845/Solo.out/adata_raw.h5ad \
    --output /mnt/data/Proyectos/kranocito/data/ARAUZO_03/results_STAR/AZ7845/Solo.out/CellBender/adata_raw_cellbender.h5 \
    --cuda

In [ ]:
adata_AZ7846.write_h5ad('data/ARAUZO_03/results_STAR/AZ7846/Solo.out/adata_raw.h5ad')

In [ ]:
!docker run --gpus all --rm -v /mnt/data:/mnt/data \
  us.gcr.io/broad-dsde-methods/cellbender:0.3.2 \
  cellbender remove-background \
    --input /mnt/data/Proyectos/kranocito/data/ARAUZO_03/results_STAR/AZ7846/Solo.out/adata_raw.h5ad \
    --output /mnt/data/Proyectos/kranocito/data/ARAUZO_03/results_STAR/AZ7846/Solo.out/CellBender/adata_raw_cellbender.h5 \
    --cuda

In [ ]:
adata_AZ7845 = load_full_adata('data/ARAUZO_03/results_STAR/AZ7845/Solo.out', include_cellbender=True)

In [ ]:
adata_AZ7845

In [ ]:
adata_AZ7846 = load_full_adata('data/ARAUZO_03/results_STAR/AZ7846/Solo.out', include_cellbender=True)

In [ ]:
adata_AZ7846

In [ ]:
adata_AZ7845.obs

In [ ]:
def plot_cell_stats(adata):
    # Show cell 10X plot (it'll be truncated, but its good enough) and relationship between this and cell probabilities
    fig, axs = plt.subplots(1, 3, figsize=(15,5))

    gene_counts = np.sort(adata.obs['n_cellbender'])[::-1]
    axs[0].plot(np.log10(np.arange(adata.shape[0]) + 1), np.log10(gene_counts + 1))

    gene_counts_filter = gene_counts[gene_counts > 200]
    axs[0].plot(np.log10(np.arange(len(gene_counts_filter)) + 1), np.log10(gene_counts_filter + 1))
    axs[0].set_xlabel('Cell rank')
    axs[0].set_ylabel('CellBender counts (log1p)')

    axs[1].scatter(adata.obs['cell_probability'], np.log10(adata.obs['n_cellbender'] + 1), s=2)
    axs[1].set_xlabel('Cell probability')
    axs[1].set_ylabel('CellBender counts (log1p)')

    axs[2].scatter(np.log10(adata.obs['n_cellbender'] + 1), np.log10(adata.obs['n_raw_counts'] + 1), s=2)
    axs[2].plot([0, 5], [0, 5], c='#bc0000')
    axs[2].set_xlabel('CellBender counts (cells)')
    axs[2].set_ylabel('Raw counts (cells)')


    plt.tight_layout()

def plot_gene_stats(adata):
    # Show ambient gene expression
    display(adata.var.sort_values(by='ambient_expression', ascending=False).head(15))

    fig, axs = plt.subplots(1, 3, figsize=(15,5))


    axs[0].scatter(np.log10(adata.var['n_cellbender'] + 1), adata.var['ambient_expression'], s=2)
    axs[0].set_xlabel('CellBender counts (genes)')
    axs[0].set_ylabel('Raw counts (log1p)')

    axs[1].scatter(np.log10(adata.var['n_cellbender'] + 1), (adata.var['n_cellbender'] + 1) / (adata.var['n_raw_counts'] + 1),  s=2)
    axs[1].set_xlabel('CellBender counts (genes)')
    axs[1].set_ylabel('$\\frac{\\text{CellBender counts  + 1}}{\\text{Raw counts + 1}}$ (genes)')

    axs[2].scatter(np.log10(adata.var['n_cellbender'] + 1), np.log10(adata.var['n_raw_counts'] + 1), s=2)
    axs[2].plot([0, 6], [0, 6], c='#bc0000')
    axs[2].set_xlabel('CellBender counts (genes)')
    axs[2].set_ylabel('Raw counts (genes)')
    plt.tight_layout()


In [ ]:
plot_cell_stats(adata_AZ7845)
plot_gene_stats(adata_AZ7845)


In [ ]:
plot_cell_stats(adata_AZ7846)
plot_gene_stats(adata_AZ7846)

In [ ]:
sc.pp.filter_cells(adata_AZ7845, min_counts=200)
sc.pp.filter_cells(adata_AZ7846, min_counts=200)

In [ ]:
adata = sc.AnnData.concatenate(adata_AZ7845, adata_AZ7846, batch_key='batch', batch_categories=['AZ7845', 'AZ7846'])

In [ ]:
(adata.layers['unspliced'].sum() + adata.layers['spliced'].sum() + adata.layers['ambiguous'].sum()) / (adata.layers['raw_counts'].sum())

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20, )

## QC - mt and raw reads

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=True, inplace=True)

In [ ]:
sc.pl.violin(adata, ['log1p_n_genes_by_counts', 'log1p_total_counts', 'pct_counts_mt'], jitter=0.4, multi_panel=True)
sc.pl.scatter(adata, x='log1p_total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='log1p_total_counts', y='log1p_n_genes_by_counts')
sc.pl.scatter(adata, x='log1p_total_counts', y='cell_probability')

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 8))
df = pd.DataFrame({'x': adata.obs['batch'], 'y': adata.obs['log1p_n_genes_by_counts'], 'y2': adata.obs['pct_counts_mt']})
sns.violinplot(x='x', y='y', data=df, ax=axs[0])
sns.violinplot(x='x', y='y2', data=df, ax=axs[1])

In [ ]:
adata.obs['lowQC_gene_counts'] = 0
adata.obs['lowQC_mt_pct'] = 0

adata.obs.loc[adata.obs['log1p_n_genes_by_counts'] < 6, 'lowQC_gene_counts'] = 1
adata.obs.loc[adata.obs['pct_counts_mt'] > 10, 'lowQC_mt_pct'] = 1

adata.obs['lowQC_gene_mt'] = adata.obs['lowQC_gene_counts'] * adata.obs['lowQC_mt_pct']

In [ ]:
adata.obs[['lowQC_gene_counts', 'lowQC_mt_pct', 'lowQC_mt_pct']].sum()

## QC - DropletQC [original workflow]

In [ ]:
%load_ext rpy2.ipython

In [ ]:
from rpy2.robjects import globalenv
# Convertimos a pandas automáticamente
from rpy2.robjects import pandas2ri

In [ ]:
adata.obs['n_spliced'] = adata.layers['spliced'].sum(axis=1).A1
adata.obs['n_unspliced'] = adata.layers['unspliced'].sum(axis=1).A1
adata.obs['n_ambiguous'] = adata.layers['ambiguous'].sum(axis=1).A1

adata.obs['nuclear_RNA_frac'] = adata.obs['n_unspliced'] / (adata.obs['n_spliced'] + adata.obs['n_unspliced'])

In [ ]:
adata.obs['n_spliced_log1p'] = np.log1p(adata.obs['n_spliced'])
adata.obs['n_unspliced_log1p'] = np.log1p(adata.obs['n_unspliced'])
adata.obs['n_ambiguous_log1p'] = np.log1p(adata.obs['n_ambiguous'])
adata.obs['n_cellbender_log1p'] = np.log1p(adata.obs['n_cellbender'])

In [ ]:
nf_df = pd.DataFrame(adata.obs['nuclear_RNA_frac'])
nf = nf_df['nuclear_RNA_frac'].values
umi = np.asarray(adata.X.sum(axis=1)).ravel()

In [ ]:
%%R -i umi -i nf_df -o ed -o dc -o plots

library(DropletQC)

nf_umi <- data.frame(
  nf = nf_df$nuclear_RNA_frac,
  umi = umi
)
# 1) Etiquetas de empty droplets
ed <- identify_empty_drops(
  nf_umi = nf_umi,
  nf_rescue = 0.02,
  umi_rescue = 3000,
  include_plot = TRUE
)

if (!"cell_type" %in% colnames(ed)) ed$cell_type <- "all"

dc <- identify_damaged_cells(
  ed,
  verbose = FALSE,
  output_plots = TRUE
)

plots <- dc[[2]]  # NULL si output_plots=FALSE
# El dataframe final con 'cell_status' actualizado está en:
dc <- dc[[1]]

## QC - DropletQC [*improved* workflow]
The workflow from DropletQC failed in two parts:
- This clusters from empty droplets in not well partitioned -> we apply a KMeans clustering between the main cell cluster and a secondary cluster with low nuclear fraction
- It does not identify a damaged cell cluster, when its likely that one exists. To solve that I run a second clustering with either a GMM or a K-Means and ensure that the damaged cell cluster has at least > 0.4 of nuclear fraction compared to the main cluster. In that way I retrieve a better clustering solution.

In [ ]:
from pyfuncs.dropletQC import classify_empty_and_damaged

In [ ]:
nf_df['umi'] = umi
nf_df['logumi'] = np.log10(umi + 1.0)

display(nf_df)

info = classify_empty_and_damaged(
    nf_df,
    method_cells="kmeans",          # o "gmm"
    nf_rescue=0.02, umi_rescue=3000,
    nf_damaged_threshold=0.0,
    random_state=0
)

info = classify_empty_and_damaged(
    nf_df,
    method_cells="gmm",          # o "gmm"
    nf_rescue=0.02, umi_rescue=3000,
    nf_damaged_threshold=0.0,
    random_state=0
)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
sns.scatterplot(x='logumi', y='nuclear_RNA_frac', hue='classification_kmeans', data=nf_df, s=2, alpha=0.5, legend=None, ax=axs[0])
sns.scatterplot(x='logumi', y='nuclear_RNA_frac', hue='classification_gmm', data=nf_df, s=2, alpha=0.5, legend=None, ax=axs[1])

In [ ]:
adata.obs['QC_droplet'] = nf_df['classification_gmm']

## QC - MALAT1 levels

In [ ]:
adata.obs['MALAT1'] = np.log1p(adata[:, 'Malat1'].X.todense().A1)

In [ ]:
sns.scatterplot(x='nuclear_RNA_frac', y='MALAT1', hue='QC_droplet', data=adata.obs, s=2, alpha=0.5)

In [ ]:
adata.obs['QC_low_MALAT1'] = 0
adata.obs.loc[(adata.obs['MALAT1'] < 2), 'QC_low_MALAT1'] = 1

In [ ]:
# Jaccard index for empty droplets and low MALAT1 levels
np.sum((adata.obs['QC_low_MALAT1'] == 1) * (adata.obs['QC_droplet'] == 'empty_droplet'))   / np.sum((adata.obs['QC_low_MALAT1'] == 1) + (adata.obs['QC_droplet'] == 'empty_droplet')) 

We see that all empty droplets have a low *Malat1* level, whereas cells with increased levels or normal levels are already detected as cell or damaged cell.

## Checking location of low QC cells

We are going to see where the marked flags are located. Supposedly, lowQC cells should create new cell types with with background gene expression. In that case, we are going to remove them to improve capability of doublet detection.

In [ ]:
adata_all_cells = adata.copy()

In [ ]:
adata_pp = adata.copy()
sc.pp.normalize_total(adata_pp)
sc.pp.log1p(adata_pp)
sc.pp.pca(adata_pp, n_comps=15, )
sce.pp.harmony_integrate(adata_pp, 'batch', max_iter_harmony=20, random_state=seed)
sc.pp.neighbors(adata_pp, n_neighbors=15, use_rep='X_pca_harmony')
sc.tl.leiden(adata_pp, resolution=0.2, key_added="groups")
sc.tl.umap(adata_pp, min_dist=0.1, random_state=seed)

In [ ]:
sc.pl.umap(adata_pp, color=['batch', 'groups'])

In [ ]:
sc.pl.umap(adata_pp, color=['batch', 'groups', 
                            'log1p_n_genes_by_counts', 'pct_counts_mt', 
                            'lowQC_gene_counts', 'lowQC_mt_pct', 'lowQC_gene_mt',
                           'QC_low_MALAT1', 'nuclear_RNA_frac', 'QC_droplet', ])

In [ ]:
sc.tl.rank_genes_groups(adata_pp, 'groups', method='wilcoxon', key_added='rank_genes_groups_wilcoxon', 
                        use_raw=False)
sc.pl.rank_genes_groups_tracksplot(adata_pp, key='rank_genes_groups_wilcoxon', 
                                   n_genes=20, sharey=False, fontsize=12, dendrogram=False, 
                                   use_raw=False,)

We see that empry droplets, which correlate with low MALAT1, low count and high mitochondrial content cells, are gathered in a separate cell type with high expression of constitutive markers show in the figure of high expressed genes. This is expected: the genes that are going to be liberated to the empty droplet will more likely be the genes with higher expression. 

There is a second set of clusters with high nuclear fraction, labeleld as damaged cells. We are not going to remove these cells so far for two reasons: their DEGs are in line with DEGs expressed by other cells passing QC; and considering that this is a single-nuclei preparation, we do expect a higher set of cells with intronic content. We may remove these cell types afterwards if we see that they don't provide any useful information.

In [ ]:
adata = adata[(adata.obs['QC_droplet'] != 'empty_droplet') &  
              (adata.obs['lowQC_gene_counts'] == 0) & 
              (adata.obs['lowQC_mt_pct'] == 0), :].copy()

In [ ]:
adata_pp = adata.copy()
sc.pp.normalize_total(adata_pp)
sc.pp.log1p(adata_pp)
sc.pp.pca(adata_pp, n_comps=15, )
sce.pp.harmony_integrate(adata_pp, 'batch', max_iter_harmony=20, random_state=seed)
sc.pp.neighbors(adata_pp, n_neighbors=15, use_rep='X_pca_harmony')
sc.tl.leiden(adata_pp, resolution=0.5, key_added="groups")
sc.tl.umap(adata_pp, min_dist=0.2, random_state=seed)

In [ ]:
sc.pl.umap(adata_pp, color=['batch', 'groups', 
                            'log1p_n_genes_by_counts', 'pct_counts_mt', 
                            'lowQC_gene_counts', 'lowQC_mt_pct', 'lowQC_gene_mt',
                           'QC_low_MALAT1', 'nuclear_RNA_frac', 'QC_droplet', ])

In [ ]:
sc.pl.umap(adata_pp, color=['batch', 'groups'])

In [ ]:
# We are going to pass this initial clustering solution to the adata, for clustering purposes
adata.obs['groups'] = adata_pp.obs['groups'].copy()
adata.obsm['X_umap'] = adata_pp.obsm['X_umap'].copy()

In [ ]:
del adata_pp
gc.collect()

## Doublet flagging

To do this step we need some clusters. To do that we are going to run a basic clustering using harmony and then use the detected cell types as input of clusters

### DoubletDetection

In [ ]:
%%script False
!pip install doubletdetection==4.3
!pip install louvain

In [ ]:
import doubletdetection

In [ ]:
clf = doubletdetection.BoostClassifier(
    n_iters=10, 
    clustering_algorithm="louvain", 
    standard_scaling=True,
    pseudocount=0.1,
    n_jobs=-1,
)
doublets = clf.fit(adata.X).predict(p_thresh=1e-16, voter_thresh=0.5)
doublet_score = clf.doublet_score()


In [ ]:
clf.doublet_score().data

In [ ]:
adata.obs['DoubletDetection_score'] = doublet_score.data
adata.obs['DoubletDetection_mask'] = doublet_score.mask

In [ ]:
f = doubletdetection.plot.convergence(clf, show=True, p_thresh=1e-16, voter_thresh=0.5)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
sns.scatterplot(x='log1p_n_genes_by_counts', y='nuclear_RNA_frac', hue='DoubletDetection_mask', data=adata.obs, s=2, alpha=0.5, ax=axs[0])
sns.scatterplot(x='log1p_n_genes_by_counts', y='nuclear_RNA_frac', hue='DoubletDetection_score', data=adata.obs, s=2, alpha=0.5, ax=axs[1])

In [ ]:
sc.pl.umap(adata, color=['DoubletDetection_mask', 'DoubletDetection_score',], cmap=magma)

### Scrublet

In [ ]:
results = sce.pp.scrublet(adata.copy(), batch_key='groups', expected_doublet_rate=0.05, random_state=seed, knn_dist_metric='cosine', log_transform=True, copy=True)
adata.uns['scrublet'] = results.uns['scrublet']
adata.obs['scrublet_score'] = results.obs['doublet_score']
adata.obs['scrublet_mask'] = results.obs['predicted_doublet']

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
sns.scatterplot(x='log1p_n_genes_by_counts', y='nuclear_RNA_frac', hue='scrublet_mask', data=adata.obs, s=2, alpha=0.5, ax=axs[0])
sns.scatterplot(x='log1p_n_genes_by_counts', y='nuclear_RNA_frac', hue='scrublet_score', data=adata.obs, s=2, alpha=0.5, ax=axs[1])

In [ ]:
sc.pl.umap(adata, color=['scrublet_mask', 'scrublet_score',], cmap=magma)

We don't see a clear Doublet pattern from both detection algorithms. DoubletDetection is more sensitive based on benchmarks, but the program does not classify any doublet; and the areas with highest scores are scattered through the FAPs, so it's likely that there are no doublets.

## Normalization

We are going to use scran, but the R version, because the py version was breaking with dependencies and seemed unreliable.

In [ ]:
%%R
library(scran)
library(BiocParallel)

In [ ]:
data_mat = adata.X.T.todense()
input_groups = adata.obs["groups"]

In [ ]:
%%R -i data_mat -i input_groups -o size_factors

size_factors = sizeFactors(
    computeSumFactors(
        SingleCellExperiment(
            list(counts=data_mat)), 
            clusters = input_groups,
            min.mean = 0.1,
            BPPARAM = MulticoreParam()
    )
)

In [ ]:
del data_mat
gc.collect()

In [ ]:
sns.displot(size_factors)
plt.axvline(1, c='red')

In [ ]:
# Save unnormalized log1p counts
adata.layers["normalization_log1p"] = sc.pp.log1p(adata.X, copy=True)

In [ ]:
# Perform the scran normalization as y = log(1 + x / size_factor)
adata.obs["size_factors"] = size_factors
scran = adata.X / adata.obs["size_factors"].values[:, None]
adata.layers["normalization_scran_log1p"] = np.log1p(scran)
adata.layers["normalization_scran_log1p"] = csr_matrix(adata.layers["normalization_scran_log1p"])

In [ ]:
scales_counts = sc.pp.normalize_total(adata, target_sum=1e6, inplace=False)
# log1p transform
adata.layers["normalization_CPM_1e6_log1p"] = sc.pp.log1p(scales_counts["X"], copy=True)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5), constrained_layout=True)

a1 = adata.layers["normalization_log1p"].sum(axis=1).A1
a2 = adata.layers["normalization_scran_log1p"].sum(axis=1).A1
a3 = adata.layers["normalization_CPM_1e6_log1p"].sum(axis=1).A1

sns.histplot(a1, ax=axes[0], bins=60, kde=True)
axes[0].set_title("log1p")

sns.histplot(a2, ax=axes[1], bins=60, kde=True)
axes[1].set_title("scran log1p")

sns.histplot(a3, ax=axes[2], bins=60, kde=True)
axes[2].set_title("CPM 1e6 log1p")

In [ ]:
adata.layers["normalization_CPM_1e6_log1p"]

# Data processing

In [ ]:
adata.X = adata.layers["normalization_scran_log1p"].copy()

In [ ]:
sc.pp.filter_genes(adata, min_counts=1)

### Checking batch effect correction (scanorama vs harmony)

In [ ]:
from scib_metrics.benchmark import Benchmarker, BioConservation, BatchCorrection

In [ ]:
sc.pp.pca(adata, random_state=seed, use_highly_variable=False, )

In [ ]:
sce.pp.harmony_integrate(adata, 'batch', random_state=seed, 
                           basis='X_pca', adjusted_basis='X_harmony')
sce.pp.scanorama_integrate(adata, 'batch', knn=int(0.5 * len(adata) ** 0.5), 
                         basis='X_pca', adjusted_basis='X_scanorama')


In [ ]:
bm = Benchmarker(
    adata,
    batch_key="batch",
    label_key="groups",
    bio_conservation_metrics=BioConservation(),
    batch_correction_metrics=BatchCorrection(bras=False, ilisi_knn=False, kbet_per_label=True, graph_connectivity=True, pcr_comparison=False),
    embedding_obsm_keys=["X_pca", "X_scanorama", "X_harmony"],
    n_jobs=6,
)
bm.benchmark()


In [ ]:
bm.plot_results_table()

df = bm.get_results(min_max_scale=False)
df

Based on these results we are going to use harmony, although both methods seem to work fine.

In [ ]:
# Feature selection
sc.pp.neighbors(adata, n_neighbors=int(0.5 * len(adata) ** 0.5), use_rep='X_harmony',
                random_state=seed, metric='correlation')
tk.tl.triku(adata, use_raw=False)

In [ ]:
sc.pp.pca(adata, random_state=seed, use_highly_variable=True)
sce.pp.harmony_integrate(adata, 'batch', random_state=seed, 
                           basis='X_pca', adjusted_basis='X_harmony', max_iter_harmony=30)
sc.pp.neighbors(adata, n_neighbors=int(0.5 * len(adata) ** 0.5), random_state=seed, metric='correlation', use_rep='X_harmony')

In [ ]:
sc.tl.umap(adata, min_dist=0.5)

In [ ]:
sc.tl.leiden(adata, resolution=0.1, key_added='leiden')
sc.tl.leiden(adata, resolution=1.5, key_added='leiden_sub')

In [ ]:
sc.pl.umap(adata, color=['leiden', 'leiden_sub', 'batch'], ncols=3, alpha=0.4, legend_loc='on data')
sc.pl.umap(adata, color=['log1p_n_genes_by_counts', 'pct_counts_mt', 'size_factors', 'QC_droplet'], cmap=magma)

In [ ]:
sc.pl.umap(adata, color=['size_factors'], cmap=magma, vmin=0.5, vmax=2.5)

So we see that after correcting for cell size (where "damaged cell" had lowest size factors), these "damaged cells" from FAPs become more "disperse" and some of them are integrated within the main cluster. 

# Kranocyte characterisation

In [ ]:
A_markers = ['Smim41', 'Col9a2', 'Dlk1', 'Shisa3',  'Saa1',  'Nipal1']
B_markers = ['Lypd2', 'Wnt6', 'Cldn1', 'Moxd1', 'Mansc4', 'Dleu7', 'Efnb3', 'Stra6', 'Sbspon',
              'Hcn4', 'Cldn22']  


In [ ]:
sc.pl.umap(adata, color=A_markers, cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata, color=B_markers, cmap=magma, use_raw=False)

In [ ]:
# The FACs was done as PDPN(+) CD31/Pecam1(-)

sc.pl.umap(adata, color=['Pdpn', 'Pecam1', 'Pdgfra', 'Tnmd', 'Lum', 'Prg4'], cmap=magma, use_raw=False)

## Analysis of major populations

In [ ]:
dict_markers_major_populations = {
    'Kranocyte': ['Rasgrp2', 'Tenm2', 'Inpp4b', 'Foxd2os', 'Malt1', 'Gfra2', 'Shisa3', 'Malt1', 'Thrsp', 'Gpld1', 'Smim41', 'Plxdc1', 
                  'Dlk1', 'Fetub', 'Saa1', 'Gria1', 'Greb1', 'Col9a2', 'Gli1', 'Cst6'], 
    'FAP': ['Ly6a', 'Nova1', 'Fstl1', 'Ifi205', 'Slfn5', 'Fbln2', 'Dpep1', 'Fn1', 'Pdgfra', 'Lbp', 'Ifi204'],
    'TNMD': ['Cilp2', 'Chad', 'Scx', 'Mkx', 'Rflnb', 'Ptx4', 'Gas2', 'Kctd1', 'Edil3', 'Col11a2', 'Sox6', 'Scube2', 'Cdh2', 'Matn4'],
    'Satellite': ['Dmd', 'Pax7', 'Chodl', 'Fgfr4', 'Notch3', 'Tenm4', 'Peg3', 'Fry'], 
    'Endothelial': ['Egfl7', 'Ptprb', 'Cdh5', 'Vwf', 'Esam', 'Cd36', 'Flt1', 'Tspan13', 'Emcn'],
}

In [ ]:
assign_cats(
    adata,
    dict_markers_major_populations,
    column_groupby='leiden_sub', 
    key_added='major_population',
    quantile_gene_sel=0.999,
    min_score=0.3, 
)

In [ ]:
sc.pl.umap(adata, color=['major_population'], cmap=magma, use_raw=False)

## Analysing Tnmd+ populations

In [ ]:
dict_markers_tnmd = {
    'TNMD_A': ['Sparcl1', 'F2r', 'Col4a2', 'Ptpre', 'Col22a1', 'Fbn2', 'Col18a1', 'Rapgef4', 'Lrrn2', 'Reln', 'Adam23',],
    'TNMD_B': ['Ccdc3', 'Bmpr1b', 'Cav1', 'Wif1', 'Kera', 'C3', 'Sema3b', 'Sema3c', 'Chrdl1', 'Grem2', 'Itga2', 'Hpgd', 'Bmp3', 'Clu'],
    'TNMD_C': ['Gpx3', 'Col8a1', 'Bicc1', 'Runx2', 'Smoc2', 'Camk4', 'Spon1', 'Wnt16', 'Slit2', 'Ostn', 'H19', 'Igf2'],
    'TNMD_D': ['Abca8a', 'Celf2', 'Mfap5', 'Col14a1', 'Dpt', 'Abca8b', 'Lum', 'Cd34']
}

In [ ]:
adata_tnmd = adata[adata.obs['major_population'] == 'TNMD'].copy()

preprocessing_adata_sub(adata_tnmd)
sc.tl.leiden(adata_tnmd, resolution=0.2, key_added='leiden_sub')
sc.tl.leiden(adata_tnmd, resolution=1.2, key_added='leiden_cellassign')
sc.tl.umap(adata_tnmd, min_dist=0.1, random_state=seed)

In [ ]:
sc.pl.umap(adata_tnmd, color=dict_markers_tnmd['TNMD_C'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_tnmd, color=['batch', 'leiden_sub', 'leiden_cellassign'])

In [ ]:
sc.tl.rank_genes_groups(adata_tnmd, groupby='leiden_sub', use_raw=False)
# sc.pl.umap(adata_tnmd, color=adata_tnmd.uns['rank_genes_groups']['names']['2'][0:150], cmap=magma, use_raw=False, ncols=5)

In [ ]:
sc.pl.umap(adata_tnmd, color=['batch', 'leiden_sub', 'QC_droplet', 
                                   'n_spliced_log1p', 'n_unspliced_log1p', 'n_ambiguous_log1p', 
                                   'nuclear_RNA_frac', 'MALAT1', 'n_cellbender_log1p'], ncols=3, 
                                   alpha=0.9, legend_loc='on data')

In [ ]:
df_pvals = plot_volcano(adata_tnmd, '0', pval_threshold=1e-3, lfc_threshold=2, topn=10, bottomn=0, return_df=True)
sc.pl.umap(adata_tnmd, color=df_pvals.sort_values(by='pvalxlfc', ascending=False)['gene'].values[:10], cmap=magma, use_raw=False, ncols=5)

In [ ]:
df_pvals = plot_volcano(adata_tnmd, '1', pval_threshold=1e-3, lfc_threshold=1, topn=10, bottomn=0, return_df=True)
sc.pl.umap(adata_tnmd, color=df_pvals.sort_values(by='pvalxlfc', ascending=False)['gene'].values[:10], cmap=magma, use_raw=False, ncols=5)

In [ ]:
df_pvals = plot_volcano(adata_tnmd, '2', pval_threshold=1e-3, lfc_threshold=2, topn=10, bottomn=0, return_df=True)
sc.pl.umap(adata_tnmd, color=df_pvals.sort_values(by='pvalxlfc', ascending=False)['gene'].values[:10], cmap=magma, use_raw=False, ncols=5)

In [ ]:
df_pvals = plot_volcano(adata_tnmd, '3', pval_threshold=1e-3, lfc_threshold=2, topn=10, bottomn=0, return_df=True)
sc.pl.umap(adata_tnmd, color=df_pvals.sort_values(by='pvalxlfc', ascending=False)['gene'].values[:10], cmap=magma, use_raw=False, ncols=5)

In [ ]:
assign_cats(
    adata_tnmd,
    dict_markers_tnmd,
    column_groupby='leiden_cellassign', 
    key_added='minor_population',
    quantile_gene_sel=0.99,
    min_score=0.3, 
)

sc.pl.umap(adata_tnmd, color=['minor_population'], cmap=magma, use_raw=False)

## Analysing Satellite populations

In [ ]:
dict_markers_satellite = {
    'SAT_A': ['Cox4i1', 'Hint1', 'Itm2a', 'Cox7a2l', 'Apoe', 'Eif3f', 'Selenof', 'Nme2', 'Ppib', 'Gapdh', 
              'Serf2', 'Myl6', 'Ost4', 'Itm2b', 'Actb', 'Tubb2b', 'Selenok', 'Use1', 'Elob', 'Ctsl', 'Drap1'],   # No clear markers
    'SAT_B': ['Dmd', 'Rora', 'Ror1', 'Esr1', 'Ext1', 'Taco1', 'Nf1', 'Ncoa2', 'Pias1', 'Nbea', 'Frem1', 'Bcl2', 'Faf1', 'Slc9a9'],
    'SAT_C': ['Bgn', 'Dcn', 'Col1a2', 'Col1a1', 'Dlc1', 'Pcolce', 'Nid1', 'Rnase4', 'Spry2', 'Serpinf1', 'Clec3b', 'Col6a3', 'Mfap5', 'Lum', 'C1s1', 'Ly6a', 'Pdgfra', 
              'Phldb1', 'Ly6c1', 'Loxl1', 'Meox2', 'Vcan', 'B4galt1', 'Svil', 'Gpc3', 'Tns1', 'Dpysl3', 
              ]
}

In [ ]:
adata_satellite = adata[adata.obs['major_population'] == 'Satellite'].copy()

preprocessing_adata_sub(adata_satellite)
sc.tl.leiden(adata_satellite, resolution=0.3, key_added='leiden_sub')
sc.tl.leiden(adata_satellite, resolution=1.2, key_added='leiden_cellassign')
sc.tl.umap(adata_satellite, min_dist=1, random_state=seed)

In [ ]:
sc.pl.umap(adata_satellite, color=['batch', 'leiden_sub', 'leiden_cellassign'])

In [ ]:
# %%script True

sc.tl.rank_genes_groups(adata_satellite, groupby='leiden_sub', use_raw=False)
# sc.pl.umap(adata_satellite, color=adata_satellite.uns['rank_genes_groups']['names']['3'][:100], cmap=magma, use_raw=False, ncols=5)

In [ ]:
sc.pl.umap(adata_satellite, color=['batch', 'leiden_sub', 'QC_droplet', 
                                   'n_spliced_log1p', 'n_unspliced_log1p', 'n_ambiguous_log1p', 
                                   'nuclear_RNA_frac', 'MALAT1', 'n_cellbender_log1p'], ncols=3, 
                                   alpha=0.9, legend_loc='on data')

In [ ]:
df_pvals = plot_volcano(adata_satellite, '0', pval_threshold=1e-20, lfc_threshold=2, topn=10, bottomn=0, return_df=True, plot_positive_only=True)
sc.pl.umap(adata_satellite, color=df_pvals.sort_values(by='pvalxlfc', ascending=False)['gene'].values[:10], cmap=magma, use_raw=False, ncols=5)

In [ ]:
df_pvals = plot_volcano(adata_satellite, '1', pval_threshold=1e-10, lfc_threshold=1, topn=10, bottomn=0, return_df=True, plot_positive_only=True)
sc.pl.umap(adata_satellite, color=df_pvals.sort_values(by='pvalxlfc', ascending=False)['gene'].values[:10], cmap=magma, use_raw=False, ncols=5)

In [ ]:
df_pvals = plot_volcano(adata_satellite, '2', pval_threshold=1e-2, lfc_threshold=1, topn=10, bottomn=0, return_df=True, plot_positive_only=True)
sc.pl.umap(adata_satellite, color=df_pvals.sort_values(by='pvalxlfc', ascending=False)['gene'].values[:10], cmap=magma, use_raw=False, ncols=5)

In [ ]:
assign_cats(
    adata_satellite,
    dict_markers_satellite,
    column_groupby='leiden_cellassign', 
    key_added='minor_population',
    quantile_gene_sel=0.99,
    min_score=0.3, 
)

sc.pl.umap(adata_satellite, color=['minor_population'], cmap=magma, use_raw=False)

## Analysing Kranocyte populations

In [ ]:
dict_markers_kranocyte = {
    'Krano_A1': ['Dpt', 'S100a10', 'Ly6a', 'Cxcl14', 'Gpx3', 'Cd302', 'Cst3', 'Rbp4'],
    'Krano_A2': ['Meg3', 'Arhgap24', 'Adatms12', 'Grm8', 'Pardb3', 'Rian', 'Nav3', 'Trps1', 'Gria1', 'Grk5', 'Taco1', 'Tspan11'],
    'Krano_B1': ['Cpe', 'Cox6c', 'Angptl7', 'Foxd1', 'Thbs4', 'Tenm2', 'Gjb5', 'Ifitm1', 'Prxl2a'],
    'Krano_B2': ['Tenm2', 'Dlc1', 'Piezo2', 'Sorsc3', 'Cnksr2', 'Phactr3', 'Col26a1', 'Tsc1', 'Lrfn2', 'Thsd4'],
    }

In [ ]:
B_markers = ['Cpe', 'Slc1a3', 'Tec', 'Tenm2', 'Piezo2', 'Kif21a', 'Foxd1', 'Mamdc2', 
                                       'Foxd2os', 'Unc13c', 'Cldn1', 
                                      'Col28a1']
A_markers = ['Sorl1', 'Spon1', 'Alpl', 'Gpld1', 'Rgs17', 'Psat1', 'Trpm6', 'Nkd1', 'Sphkap', 
                                       'Dlk1', 'Thrsp',]


In [ ]:
adata_kranocyte = adata[adata.obs['major_population'] == 'Kranocyte'].copy()

preprocessing_adata_sub(adata_kranocyte)
sc.tl.leiden(adata_kranocyte, resolution=0.15, key_added='leiden_sub')
sc.tl.leiden(adata_kranocyte, resolution=1.2, key_added='leiden_cellassign')
sc.tl.umap(adata_kranocyte, min_dist=0.1, random_state=seed)

In [ ]:
sc.tl.leiden(adata_kranocyte, resolution=0.2, key_added='leiden_sub')
sc.tl.leiden(adata_kranocyte, resolution=1.2, key_added='leiden_cellassign')
sc.tl.umap(adata_kranocyte, min_dist=0.1, random_state=seed)

In [ ]:
sc.pl.umap(adata_kranocyte, color=['batch', 'leiden_sub', 'leiden_cellassign'])

In [ ]:
sc.tl.rank_genes_groups(adata_kranocyte, groupby='leiden_sub', use_raw=False)
# sc.pl.umap(adata_kranocyte, color=adata_kranocyte.uns['rank_genes_groups']['names']['1'][:100], cmap=magma, use_raw=False, ncols=5)

In [ ]:
sc.pl.umap(adata_kranocyte, color=['batch', 'leiden_sub', 'QC_droplet', 
                                   'n_spliced_log1p', 'n_unspliced_log1p', 'n_ambiguous_log1p', 
                                   'nuclear_RNA_frac', 'MALAT1', 'n_cellbender_log1p'], ncols=3, 
                                   alpha=0.9, legend_loc='on data')

In [ ]:
df_pvals = plot_volcano(adata_kranocyte, '0', pval_threshold=1e-10, lfc_threshold=2, topn=10, bottomn=0, return_df=True, plot_positive_only=True)
sc.pl.umap(adata_kranocyte, color=df_pvals.sort_values(by='pvalxlfc', ascending=False)['gene'].values[:20], cmap=magma, use_raw=False, ncols=5)

In [ ]:
df_pvals = plot_volcano(adata_kranocyte, '1', pval_threshold=1e-3, lfc_threshold=1, topn=20, bottomn=0, return_df=True, plot_positive_only=True)
sc.pl.umap(adata_kranocyte, color=df_pvals.sort_values(by='pvalxlfc', ascending=False)['gene'].values[:10], cmap=magma, use_raw=False, ncols=5)

In [ ]:
df_pvals = plot_volcano(adata_kranocyte, '2', pval_threshold=1e-5, lfc_threshold=1, topn=10, bottomn=0, return_df=True, plot_positive_only=True)
sc.pl.umap(adata_kranocyte, color=df_pvals.sort_values(by='pvalxlfc', ascending=False)['gene'].values[:10], cmap=magma, use_raw=False, ncols=5)

In [ ]:
df_pvals = plot_volcano(adata_kranocyte, '3', pval_threshold=0.05, lfc_threshold=1, topn=20, bottomn=0, return_df=True, plot_positive_only=True)
sc.pl.umap(adata_kranocyte, color=df_pvals.sort_values(by='pvalxlfc', ascending=False)['gene'].values[:10], cmap=magma, use_raw=False, ncols=5)

In [ ]:
assign_cats(
    adata_kranocyte,
    dict_markers_kranocyte,
    column_groupby='leiden_cellassign', 
    key_added='minor_population',
    quantile_gene_sel=0.99,
    min_score=0.3, 
)

sc.pl.umap(adata_kranocyte, color=['minor_population'], cmap=magma, use_raw=False)

## Analysing FAP populations

In [ ]:
dict_markers_FAP = {
    'FAP_A1': ['Pi16', 'Dact2', 'Uchl1', 'Efemp1', 'Car8', 'Cmah', 'Il18', 'Stmn4', 'Efhd1', 'Sbsn', ],
    'FAP_A2': ['Sdk1', 'Opcml', 'Ppp2r2b', 'Slc4a4', 'Efna5', 'Ang', 'Adamts16', 'Taco1', 'Pdgfd', 'Rorb'], # Not good quality
    'FAP_B1': ['Cxcl14', 'Hsd11b1', 'Smoc2', 'Col15a1', 'Mme', 'G0s2', 'Col4a1', 'Lamb1', 'Col4a2', 
               'Cldn15', 'Clec14a', 'Nmb', 'Vwa1', 'Crlf1',],
    'FAP_B2': ['Grm8', 'Rora', 'Plxdc2', 'Ank2', 'Pcdh7', 'Eda', 'Tnik', 'Adgrl3', 'Elmo1'],
    'FAP_C': ['Gdf10', 'C2', 'Cygb', 'Fmo2', 'Nr2f2', 'Csmd1', 'Prdm6', 'Abcc9', 'Gria4', 'Clec11a', 'Ism1', 
               'Gata6', 'Pltp', 'Hhip', 'Tspan11', 'Tent5c'],
    'FAP_D': ['Steap4', 'Apoe', 'Cyp1b1', 'Aoc3', 'Ccl19-ps4', 'Sfrp1', 'Adam12', 'Emb', 
               'Agt', 'Pparg',  ],
    'FAP_E': ['Mgp', 'Hmcn1', 'Meox1', 'Meox2', 'Clu', 'Etl4', 'Kctd12', 'Boc', 'Daam2', 'Matn2', 
              'Robo2', 'Clec1a', 'Myo10', 'Sgip1', 'Myo1b', 'Ptn', 'Mettl24', 'Rgs7bp'],
    'FAP_F': ['Thbs2', 'Gpx3', 'Cilp', 'Fgl2', 'Lox', 'Aspn', 'Ccdc80', 'Igfbp3', 'Dkk2', 'Ecrg4', 'Tnmd', 
              'Col12a1', 'Pappa2', 'Egfl6', 'Fibin', 'Ccn5', 'Sfrp4', 'Fxyd6', ],
    }

In [ ]:
adata_FAP = adata[adata.obs['major_population'] == 'FAP'].copy()

preprocessing_adata_sub(adata_FAP)
sc.tl.leiden(adata_FAP, resolution=0.15, key_added='leiden_sub')
sc.tl.leiden(adata_FAP, resolution=1.2, key_added='leiden_cellassign')
sc.tl.umap(adata_FAP, min_dist=0.1, random_state=seed)

In [ ]:
sc.tl.leiden(adata_FAP, resolution=0.6, key_added='leiden_sub')
sc.tl.leiden(adata_FAP, resolution=1.2, key_added='leiden_cellassign')
sc.tl.umap(adata_FAP, min_dist=0.1, random_state=seed)

In [ ]:
sc.pl.umap(adata_FAP, color=['batch', 'leiden_sub', 'leiden_cellassign'])

In [ ]:
# %%script True

sc.tl.rank_genes_groups(adata_FAP, groupby='leiden_sub', use_raw=False)
# sc.pl.umap(adata_FAP, color=adata_FAP.uns['rank_genes_groups']['names']['1'][:100], cmap=magma, use_raw=False, ncols=5)

In [ ]:
sc.pl.umap(adata_FAP, color=['batch', 'leiden_sub', 'QC_droplet', 
                                   'n_spliced_log1p', 'n_unspliced_log1p', 'n_ambiguous_log1p', 
                                   'nuclear_RNA_frac', 'MALAT1', 'n_cellbender_log1p'], ncols=3, 
                                   alpha=0.9, legend_loc='on data')

In [ ]:
df_pvals = plot_volcano(adata_FAP, '0', pval_threshold=1e-50, lfc_threshold=1, topn=30, bottomn=0, return_df=True, plot_positive_only=True)
sc.pl.umap(adata_FAP, color=['leiden_sub'] + df_pvals.sort_values(by='pvalxlfc', ascending=False)['gene'].values[:14].tolist(), cmap=magma, use_raw=False, ncols=5)

In [ ]:
df_pvals = plot_volcano(adata_FAP, '3', pval_threshold=1e-50, lfc_threshold=1, topn=30, bottomn=0, return_df=True, plot_positive_only=True)
sc.pl.umap(adata_FAP, color=['leiden_sub'] + df_pvals.sort_values(by='pvalxlfc', ascending=False)['gene'].values[:14].tolist(), cmap=magma, use_raw=False, ncols=5)

In [ ]:
df_pvals = plot_volcano(adata_FAP, '9', pval_threshold=1e-40, lfc_threshold=1, topn=20, bottomn=0, return_df=True, plot_positive_only=True)
sc.pl.umap(adata_FAP, color=['leiden_sub'] + df_pvals.sort_values(by='pvalxlfc', ascending=False)['gene'].values[:14].tolist(), cmap=magma, use_raw=False, ncols=5)

In [ ]:
df_pvals = plot_volcano(adata_FAP, '1', pval_threshold=1e-50, lfc_threshold=1, topn=50, bottomn=0, return_df=True, plot_positive_only=True)
sc.pl.umap(adata_FAP, color=['leiden_sub'] + df_pvals.sort_values(by='pvalxlfc', ascending=False)['gene'].values[:14].tolist(), cmap=magma, use_raw=False, ncols=5)

In [ ]:
df_pvals = plot_volcano(adata_FAP, '4', pval_threshold=1e-50, lfc_threshold=1, topn=50, bottomn=0, return_df=True, plot_positive_only=True)
sc.pl.umap(adata_FAP, color=['leiden_sub'] + df_pvals.sort_values(by='pvalxlfc', ascending=False)['gene'].values[:14].tolist(), cmap=magma, use_raw=False, ncols=5)

In [ ]:
df_pvals = plot_volcano(adata_FAP, '7', pval_threshold=1e-50, lfc_threshold=1, topn=50, bottomn=0, return_df=True, plot_positive_only=True)
sc.pl.umap(adata_FAP, color=['leiden_sub'] + df_pvals.sort_values(by='pvalxlfc', ascending=False)['gene'].values[:14].tolist(), cmap=magma, use_raw=False, ncols=5)

In [ ]:
df_pvals = plot_volcano(adata_FAP, '6', pval_threshold=1e-30, lfc_threshold=1, topn=20, bottomn=0, return_df=True, plot_positive_only=True)
sc.pl.umap(adata_FAP, color=['leiden_sub'] + df_pvals.sort_values(by='pvalxlfc', ascending=False)['gene'].values[:14].tolist()
, cmap=magma, use_raw=False, ncols=5)

In [ ]:
df_pvals = plot_volcano(adata_FAP, '5', pval_threshold=1e-50, lfc_threshold=1, topn=10, bottomn=0, return_df=True, plot_positive_only=True)
sc.pl.umap(adata_FAP, color=['leiden_sub'] + df_pvals.sort_values(by='pvalxlfc', ascending=False)['gene'].values[:14].tolist(), cmap=magma, use_raw=False, ncols=5)

In [ ]:
df_pvals = plot_volcano(adata_FAP, '2', pval_threshold=1e-50, lfc_threshold=1, topn=20, bottomn=0, return_df=True, plot_positive_only=True)
sc.pl.umap(adata_FAP, color=['leiden_sub'] + df_pvals.sort_values(by='pvalxlfc', ascending=False)['gene'].values[:14].tolist(), cmap=magma, use_raw=False, ncols=5)

In [ ]:
df_pvals = plot_volcano(adata_FAP, '8', pval_threshold=1e-50, lfc_threshold=1, topn=10, bottomn=0, return_df=True, plot_positive_only=True)
sc.pl.umap(adata_FAP, color=['leiden_sub'] + df_pvals.sort_values(by='pvalxlfc', ascending=False)['gene'].values[:14].tolist(), cmap=magma, use_raw=False, ncols=5)

In [ ]:
assign_cats(
    adata_FAP,
    dict_markers_FAP,
    column_groupby='leiden_cellassign', 
    key_added='minor_population',
    quantile_gene_sel=0.7,
    min_score=0.3, 
)

sc.pl.umap(adata_FAP, color=['minor_population'], cmap=magma, use_raw=False)

## Analysis of endothelial subpopulations (Not applied because of too few cells)

In [ ]:
adata_endo = adata[adata.obs['major_population'] == 'Endothelial'].copy()

preprocessing_adata_sub(adata_endo, integrate=False, k=5)
sc.tl.leiden(adata_endo, resolution=0.15, key_added='leiden_sub')
sc.tl.leiden(adata_endo, resolution=1.2, key_added='leiden_cellassign')
sc.tl.umap(adata_endo, min_dist=0.1, random_state=seed)

In [ ]:
sc.tl.leiden(adata_endo, resolution=0.4, key_added='leiden_sub')
sc.tl.leiden(adata_endo, resolution=1.2, key_added='leiden_cellassign')
sc.tl.umap(adata_endo, min_dist=0.1, random_state=seed)

In [ ]:
sc.pl.umap(adata_endo, color=['batch', 'leiden_sub', 'leiden_cellassign'], s=200)

In [ ]:
adata_endo.obs['minor_population'] = 'Endothelial'

## Applying minor cell types in general adata

In [ ]:
adata.obs['minor_population'] = ''

for adata_sub in [adata_FAP, adata_kranocyte, adata_satellite, adata_tnmd, adata_endo]:
    adata.obs.loc[adata_sub.obs_names, 'minor_population'] = adata_sub.obs['minor_population'].values

In [ ]:
sc.pl.umap(adata, color=['major_population', 'minor_population'])

# TODO: PSUEDOTEMPORAL
## PHATE + PAGA
## PHATE + VELOCYTO
## PHATE + MONOCLE

# Preparing adata for cellxgene

In [ ]:
adata_cellxgene = adata.copy()

In [ ]:
adata_cellxgene.uns['schema_version'] = '3.0.0'
adata_cellxgene.uns['title'] = 'Analysis of muscle fibroblast heterogeneity'
adata_cellxgene.uns['batch_condition'] = ['sample']
adata_cellxgene.uns['default_embedding'] = 'X_umap'

In [ ]:
columns_in = ['scrublet', 'log1p', 'pca', 'neighbors', 'triku_params', 'umap', 'cell_assign', 'schema_version', 
'title', 'batch_condition', 'default_embedding']
columns_out = [i for i in adata_cellxgene.uns.keys() if i not in columns_in ]

print('Columns out: ', columns_out)

for i in columns_out:
    del adata_cellxgene.uns[i]

In [ ]:
adata_cellxgene.obs['organism_ontology_term_id'] = 'NCBITaxon:10090'
adata_cellxgene.obs['tissue_ontology_term_id'] = 'UBERON:0001134'
adata_cellxgene.obs['assay_ontology_term_id'] = 'EFO:0009922' # 10x 3' v3
adata_cellxgene.obs['disease_ontology_term_id'] = 'PATO:0000461'
adata_cellxgene.obs['cell_type_ontology_term_id'] = 'CL:1001609'
adata_cellxgene.obs['self_reported_ethnicity_ontology_term_id'] = 'na'
adata_cellxgene.obs['development_stage_ontology_term_id'] = 'MmusDv:0000063'
adata_cellxgene.obs['sex_ontology_term_id'] = 'PATO:0000384'
adata_cellxgene.obs['donor_id'] = adata_cellxgene.obs['sample']
adata_cellxgene.obs['suspension_type'] = 'cell'


In [ ]:
adata_cellxgene.obs = adata_cellxgene.obs.rename(columns={'assigned_cats': 'author_cell_type', 'leiden_assignedcats': 'leiden'})

accepted_cats = ['sample', 'log1p_n_genes_by_counts', 'log1p_total_counts', 'pct_counts_mt',
                'author_cell_type', 'organism_ontology_term_id', 
                'tissue_ontology_term_id', 'assay_ontology_term_id', 'disease_ontology_term_id', 'cell_type_ontology_term_id', 
                'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'donor_id', 
                'suspension_type']
  
adata_cellxgene.obs = adata_cellxgene.obs[accepted_cats]

for cat in accepted_cats:
    if cat not in ['log1p_n_genes_by_counts', 'log1p_total_counts', 'pct_counts_mt']:
        adata_cellxgene.obs[cat] = adata_cellxgene.obs[cat].astype('category')


In [ ]:
for var in ["highly_variable", "triku_highly_variable", "mt"]:
    del adata_cellxgene.var[var]

In [ ]:
del adata_cellxgene.obsm['X_triku']
del adata_cellxgene.varm
del adata_cellxgene.obsp

In [ ]:
adata_cellxgene.var_names = adata.var['ensemble_ID']

In [ ]:
adata_cellxgene.X = adata_cellxgene.X.astype(np.float32)

In [ ]:
adata_cellxgene.raw.var.set_index('ensemble_ID', inplace=True)
adata_cellxgene.raw.var.drop(columns=['gene_symbol'], inplace=True)

In [ ]:
genes_add = [e for e in adata_cellxgene.raw.var.index if e not in adata_cellxgene.var.index]

new_matrix = sparse.csr_matrix((adata_cellxgene.X.data, adata_cellxgene.X.indices, adata_cellxgene.X.indptr), shape = adata_cellxgene.raw.shape)

all_genes = adata_cellxgene.var.index.to_list()
all_genes.extend(genes_add)

new_var = pd.DataFrame(index=all_genes)
new_var = pd.merge(new_var, adata_cellxgene.var, left_index=True, right_index=True, how='left')
new_var.loc[genes_add, 'feature_is_filtered'] = True

new_adata = ad.AnnData(X=new_matrix, dtype=new_matrix.dtype, 
                        obs=adata_cellxgene.obs, var=new_var, uns=adata_cellxgene.uns, obsm=adata_cellxgene.obsm, raw = adata_cellxgene.raw)
new_adata = new_adata[:,adata_cellxgene.raw.var.index.to_list()]
new_adata.var.loc[adata_cellxgene.var.index, 'feature_is_filtered'] = False
new_adata.var['feature_is_filtered'] = new_adata.var['feature_is_filtered'].astype(bool)



In [ ]:
adata_cellxgene = new_adata

In [ ]:
adata_cellxgene.write_h5ad('data/ARAUZO_03/20230623/output_nfcore/alevin/mtx_conversions/combined_matrix_cellxgene.h5ad', 
                 compression='gzip')

# Exporting HTML

In [ ]:
!jupyter nbconvert --to html /data/Proyectos/kranocito/4_Analysis_of_krano_dataset.ipynb

In [ ]:
    Preprocessing (this should be done with each sample individually): 

    Normalization: compute s-factors using scran and divide counts >> .obs[s_factor],  .layers[s_counts] 

    Calculate normalized metrics 

    Apply log1p transform (log y/s + 1) >> .layer[log_counts] (the default one) 

    RPM based on y/s >> .layer[RPM] 

    Pearson residuals. >> .layer[pearson_residuals] 

    Sample integration (harmony or scanorama) 

    Feature selection using Triku 

    PCA 

    kNN with cosine  

    UMAP >> Remove low QC cells and exclude clusters driven by technical artefacts 

    Rerun PCA -> UMAP to ensure that removal of lowQC clusters did not affect overall dataset structure. 

    Dataset QC metrics captured (metrics per dataset): 

    STAR alignment metrics (% mapped) 

    10X QC metrics  

    Percent of low QC cells (%ambient_rna, low MALAT, DropletQC, doublet, etc.) 